# Load related library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)


In [2]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

# Load dataset

In [3]:
trainSet = pd.read_csv("./Data/train.csv", header=0)

# Data cleaning

## Drop useless columns

In [4]:
train = trainSet.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])

## Drop rows with missing values

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  int64  
 1   Pclass    100000 non-null  int64  
 2   Sex       100000 non-null  object 
 3   Age       96708 non-null   float64
 4   SibSp     100000 non-null  int64  
 5   Parch     100000 non-null  int64  
 6   Fare      99866 non-null   float64
 7   Embarked  99750 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 6.1+ MB


In [6]:
train = train.dropna(axis=0)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96332 entries, 2 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  96332 non-null  int64  
 1   Pclass    96332 non-null  int64  
 2   Sex       96332 non-null  object 
 3   Age       96332 non-null  float64
 4   SibSp     96332 non-null  int64  
 5   Parch     96332 non-null  int64  
 6   Fare      96332 non-null  float64
 7   Embarked  96332 non-null  object 
dtypes: float64(2), int64(4), object(2)
memory usage: 6.6+ MB


## Dummy coding of the categorical variables

In [8]:
train = pd.get_dummies(train)

# Train, validation data set split

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns="Survived", axis=0)
                                                , train['Survived']
                                                , test_size=0.2
                                                , random_state=2023
                                                , stratify=train['Survived'])

# Tuning hyperparameters (**NOT including `layers`**)

- `Optimizer`
- `Activation`
- `Neurons`
- `Batch size`
- `Epochs`

## Create function

In [14]:
# Create function
def nn_cl_bo1(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

## Set parameters

In [11]:
params_nn = {
    'neurons': (10, 100)
  , 'activation': (0, 9) # `activationL` has 10 elements
  , 'optimizer': (0, 7) # `optimizerL` has 8 elements
  , 'learning_rate': (0.01, 1)
  , 'batch_size': (200, 1000)
  , "epochs": (20, 100)
}

## Run Bayesian Optimization

In [12]:
nn_bo1 = BayesianOptimization(nn_cl_bo1, params_nn, random_state=2023)
nn_bo1.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------


2023-02-11 10:28:22.977074: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


| 1         | 0.7669    | 2.898     | 912.3     | 67.04     | 0.1353    | 22.72     | 3.275     |
| 2         | 0.7665    | 0.1988    | 781.8     | 61.95     | 0.5495    | 51.07     | 3.51      |
| 3         | 0.6888    | 3.55      | 320.9     | 48.87     | 0.1705    | 40.42     | 1.262     |
| 4         | 0.7681    | 3.519     | 228.5     | 65.19     | 0.2114    | 38.85     | 2.636     |
| 5         | 0.7221    | 1.656     | 283.2     | 56.39     | 0.2039    | 44.07     | 6.514     |
| 6         | 0.5719    | 6.841     | 816.6     | 67.74     | 0.7937    | 82.93     | 6.864     |
| 7         | 0.7497    | 7.963     | 287.8     | 85.58     | 0.3145    | 33.53     | 2.84      |
| 8         | 0.579     | 4.981     | 700.4     | 26.3      | 0.9726    | 47.02     | 5.052     |
| 9         | 0.7647    | 5.97      | 374.6     | 34.97     | 0.7325    | 87.7      | 2.742     |
| 10        | 0.7608    | 0.9904    | 930.2     | 48.56     | 0.4188    | 26.52     | 4.102     |
| 11        | 0.758 

## Best hyperparameters

In [13]:
params_nn_1 = nn_bo1.max['params']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_1['activation'] = activationL[round(params_nn_1['activation'])]

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
params_nn_1['optimizer'] = optimizerL[round(params_nn_1['optimizer'])]

params_nn_1['bach_size'] = round(params_nn_1['batch_size'])
params_nn_1['epochs'] = round(params_nn_1['epochs'])

params_nn_1

{'activation': 'tanh',
 'batch_size': 228.51856832327053,
 'epochs': 65.18893191347787,
 'learning_rate': 0.21142687695433024,
 'neurons': 38.85440118990134,
 'optimizer': 'Adadelta'}

# Tuning hyperparameters (**Including `layers`**)

- `Optimizer`
- `Activation`
- `Neurons`
- `Batch size`
- `Epochs`
- `Layers`

In [18]:
# Create function
def nn_cl_bo2(neurons, activation, optimizer, learning_rate,  batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())

        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))

        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=2023))

        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))

        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [17]:
params_nn2 ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}

In [20]:
nn_bo2 = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=2023)
nn_bo2.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.7671    | 2.898     | 912.3     | 0.5881    | 0.03798   | 31.31     | 1.936     | 1.044     | 0.73      | 57.19     | 0.5449    | 3.195     |
| 2         | 0.7444    | 4.512     | 515.6     | 0.1512    | 0.1083    | 32.97     | 1.676     | 1.361     | 0.3971    | 13.21     | 0.5649    | 1.424     |
| 3         | 0.7518    | 2.885     | 501.3     | 0.1841    | 0.03119   | 56.39     | 1.392     | 1.757     | 0.9312    | 78.41     | 0.7708    | 4.177     |
| 4         | 0.5719    | 7.125     | 848.3     | 0.9806    | 0.2654    | 28.78     | 2.639     | 1.615     | 0.2689    | 46.52     | 0.5534    | 4.379     |
| 5         | 0.7041    | 0.7088    | 977.8     | 0.

In [21]:
params_nn_2 = nn_bo2.max['params']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_2['activation'] = activationL[round(params_nn_2['activation'])]

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
params_nn_2['optimizer'] = optimizerL[round(params_nn_2['optimizer'])]

params_nn_2['bach_size'] = round(params_nn_2['batch_size'])
params_nn_2['epochs'] = round(params_nn_2['epochs'])
params_nn_2['layers1'] = round(params_nn_2['layers1'])
params_nn_2['layers2'] = round(params_nn_2['layers2'])
params_nn_2['dropout'] = round(params_nn_2['dropout'])
params_nn_2['normalization'] = round(params_nn_2['normalization'])

params_nn_2

{'activation': 'softsign',
 'batch_size': 912.3379614315846,
 'dropout': 1,
 'dropout_rate': 0.03797882805128737,
 'epochs': 31,
 'layers1': 2,
 'layers2': 1,
 'learning_rate': 0.7300019655294788,
 'neurons': 57.19486080450004,
 'normalization': 1,
 'optimizer': 'Adadelta',
 'bach_size': 912}